In [ ]:
%matplotlib inline

# Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
url_format = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/{}/netcdf/A5B2/1.00/woa13_A5B2_{}{:02}_01v2.nc'

In [ ]:
temp_urls = [url_format.format('temperature', 't', i+1) for i in range(12)]
salt_urls = [url_format.format('salinity', 's', i+1) for i in range(12)]

Now we define a function that gives us a potential density section from a given month (default January), and range of latitudes and longitudes.

In [ ]:
def sect(lat, lon, month=0):
    salt_data = Dataset(salt_urls[month])
    temp_data = Dataset(temp_urls[month])
    
    depth = salt_data.variables['depth'][:]
    # calculate indices for lat/lon
    lat_d = salt_data.variables['lat'][:]
    lon_d = salt_data.variables['lon'][:]
    lat_i = (lat.min() <= lat_d) & (lat_d <= lat.max())
    lon_i = (lon.min() <= lon_d) & (lon_d <= lon.max())
    
    # compute absolute salinity from practical salinity
    sp = salt_data.variables['s_an'][0,:,lat_i,lon_i]
    print(sp.shape)
    sa = gsw.SA_from_SP(sp, depth, lon, lat)
    print(sa.shape, sa)
    
    # compute conservative temperature from in-situ temperature and absolute salinity
    ct = gsw.CT_from_t(sa, temp_data.variables['t_an'][0,:,lat_i,lon_i], depth)
    
    salt_data.close()
    temp_data.close()
    
    # compute potential density wrt surface
    return gsw.rho(sa, ct, 0)

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.

## Denmark Strait
Because it's pretty easy to find, we'll look at the Denmark strait from 22.5 to 39.5 degrees West, at 63.5 degrees North.

In [ ]:
temp = Dataset('data/woa13_A5B2_t01_01v2.nc', 'r')
salt = Dataset('data/woa13_A5B2_s01_01v2.nc', 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_i = (-39.5 <= lon) & (lon <= -22.5)

t_sect = temp.variables['t_an'][0,:,lat==63.5,lon_i].squeeze()
s_sect = salt.variables['s_an'][0,:,lat==63.5,lon_i].squeeze()
lon_sect = lon[lon_i]

Using TEOS-10, we can convert from practical salinity to absolute salinity, and from in-situ temperature to conservative temperature. From here, we can compute the locally-referenced density, and the potential density referenced to 2000m.

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, -39.5 + i, 63.5)
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

Let's take a look at the two density sections. The locally-refenced density shows very linear stratification (this is probably expected?), whereas the potential density shows a much clearer mixed layer (also as expected).

In [ ]:
ax = plt.subplot(121)
plt.pcolormesh(lon_sect, dep, rho_sect)
ax.invert_yaxis()
#plt.colorbar()

ax = plt.subplot(122)
plt.pcolormesh(lon_sect, dep, rhop_sect)
ax.invert_yaxis()
plt.colorbar()

We're looking at January, so this should be a Winter mixed layer. We'll call it about 400m deep for the purposes of calculating a grid diffusivity coefficient later on.

In [ ]:
plt.plot(rhop_sect[:,5], dep, '*')
plt.gca().invert_yaxis()

To give some information between water columns, we want to compute neutral density differences. We can use the thermal expansion and haline contraction coefficients from the mean salinity, temperature and pressure of the water parcels, as well as the local density from the mean.

In [ ]:
rhop_sect[30,5:7]

In [ ]:
sa_c = (sa_sect[30,5] + sa_sect[30,6]) / 2
ct_c = (ct_sect[30,5] + ct_sect[30,6]) / 2

In [ ]:
rho_c, alpha_c, beta_c = gsw.rho_alpha_beta(sa_c, ct_c, dep[30])

In [ ]:
nd_diff = rho_c * (beta_c * (sa_sect[30,5] - sa_sect[30,6]) - alpha_c * (ct_sect[30,5] - ct_sect[30,6]))
print(nd_diff)

Now we want to compute a distance (in physical space) by which to shift the parcel in order to flatten out the difference.

In [ ]:
g = 9.7963 # value of gravity used in gsw

In [ ]:
n2 = gsw.Nsquared(sa_sect[29:31,5].mean(),
                  ct_sect[29:31,5].mean(),
                  dep[29:31])
dz = (g**2 * nd_diff) / (1e4 * n2[1])
print(dz)